## Employee Attrition 



In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from collections import Counter
import pandas_profiling
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.svm import SVC
import plotly.express as px
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [21]:
train = pd.read_csv('/Users/mac/Desktop/group_assignment_ML2/data/train.csv', sep = ",")
test = pd.read_csv('/Users/mac/Desktop/group_assignment_ML2/data/test.csv', sep = ",")

In [37]:
X= train.drop(["Attrition"],axis=1)
y = train["Attrition"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=99)

Sc = StandardScaler()
ohe = OneHotEncoder()
numeric_features = ["Age","DailyRate","DistanceFromHome","Education","HourlyRate","JobLevel","MonthlyIncome","MonthlyRate","NumCompaniesWorked","PercentSalaryHike",
                    "TotalWorkingYears","TrainingTimesLastYear","YearsAtCompany","YearsInCurrentRole","YearsSinceLastPromotion","YearsWithCurrManager"]
categorical_features = ["BusinessTravel","Gender","Department","MaritalStatus","EducationField","JobRole","OverTime","WorkLifeBalance"]
preprocess = ColumnTransformer(
    transformers=[
                ('num', Sc, numeric_features),
                ('cat', ohe, categorical_features)])

models=[LogisticRegression(solver="liblinear",max_iter=1000),RandomForestClassifier(),GradientBoostingClassifier(random_state=99,learning_rate=0.05,max_depth=2,n_estimators=200),xgb.XGBClassifier(random_state=99,learning_rate=0.05,max_depth=2,n_estimators=200)]


auc=[]

for x in models:
    model = Pipeline(steps=[('preprocess', preprocess),
                            ('model', x)])
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    auc.append(roc_auc)

model_names = ['Logistic Regression', 'Random Forest','Gradient Boosting',"XGBClassifier"]
scores= pd.DataFrame({'model_names': model_names,
                      "auc":auc})
scores

,model_names,auc
0,Logistic Regression,0.811115
1,Random Forest,0.842245
2,Gradient Boosting,0.865791
3,XGBClassifier,0.866999


In [23]:
y_pred = model.predict(X_test)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print("auc: ",roc_auc) 

auc:  0.8669991260089456


In [24]:
# Make predictions on the test data
y_pred_test = model.predict(test)
Counter(y_pred_test)

Counter({0: 1092, 1: 27})